In [ ]:
import pandas as pd
file = "/Users/k.abhishek/Documents/customer_clusters/customer_onboard_data.xlsx"
df = pd.read_excel(file)
print(df.columns)
colNum_to_colName = {i:col for i,col in enumerate(df.columns)}
print(colNum_to_colName)

Index(['SITE_CLUSTER_ID', 'LEGAL_CLUSTER_ID', 'customer_cmd_code',
       'customer_role_type', 'trading_name', 'street_no', 'address_line1',
       'pobox', 'address_line2', 'address_line3', 'district', 'city',
       'postal_code', 'state_code', 'state_name', 'country_code',
       'country_name', 'phone_type', 'isd_country_code', 'isd_dialing_code',
       'extension_number', 'phone_num', 'create_time', 'customer_cmd_code.1',
       'BVD', 'BVD_HQ', 'BVD_GUO', 'TAXNO1', 'TAXNO2', 'TAXNO3', 'TAXNO4',
       'TAXNO5', 'VATREG'],
      dtype='object')
{0: 'SITE_CLUSTER_ID', 1: 'LEGAL_CLUSTER_ID', 2: 'customer_cmd_code', 3: 'customer_role_type', 4: 'trading_name', 5: 'street_no', 6: 'address_line1', 7: 'pobox', 8: 'address_line2', 9: 'address_line3', 10: 'district', 11: 'city', 12: 'postal_code', 13: 'state_code', 14: 'state_name', 15: 'country_code', 16: 'country_name', 17: 'phone_type', 18: 'isd_country_code', 19: 'isd_dialing_code', 20: 'extension_number', 21: 'phone_num', 22: 'create_time', 23: 'customer_cmd_code.1', 24: 'BVD', 25: 'BVD_HQ', 26: 'BVD_GUO', 27: 'TAXNO1', 28: 'TAXNO2', 29: 'TAXNO3', 30: 'TAXNO4', 31: 'TAXNO5', 32: 'VATREG'}


In [1]:
#load data from csv
import csv

with open('export.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    data = list(csvFile)

In [134]:
print(data[0])
colNum_to_colName = {i:col for i,col in enumerate(data[0])}
print(colNum_to_colName)

['SITE_CLUSTER_ID', 'LEGAL_CLUSTER_ID', 'customer_cmd_code', 'customer_role_type', 'trading_name', 'street_no', 'address_line1', 'pobox', 'address_line2', 'address_line3', 'district', 'city', 'postal_code', 'state_code', 'state_name', 'country_code', 'country_name', 'phone_type', 'isd_country_code', 'isd_dialing_code', 'extension_number', 'phone_num', 'create_time', 'customer_cmd_code', 'BVD', 'BVD_HQ', 'BVD_GUO', 'TAXNO1', 'TAXNO2', 'TAXNO3', 'TAXNO4', 'TAXNO5', 'VATREG']
{0: 'SITE_CLUSTER_ID', 1: 'LEGAL_CLUSTER_ID', 2: 'customer_cmd_code', 3: 'customer_role_type', 4: 'trading_name', 5: 'street_no', 6: 'address_line1', 7: 'pobox', 8: 'address_line2', 9: 'address_line3', 10: 'district', 11: 'city', 12: 'postal_code', 13: 'state_code', 14: 'state_name', 15: 'country_code', 16: 'country_name', 17: 'phone_type', 18: 'isd_country_code', 19: 'isd_dialing_code', 20: 'extension_number', 21: 'phone_num', 22: 'create_time', 23: 'customer_cmd_code', 24: 'BVD', 25: 'BVD_HQ', 26: 'BVD_GUO', 27: 'T

In [ ]:
import csv

customer_cmd_code_to_bvd = {}
# with open('export.csv', mode ='r') as file:
#     csvFile = csv.reader(file)
#     data = list(csvFile)

for idx, arr in enumerate(data[1:]):
    tax = set()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            # print(type(arr[i]))
            tax.add(arr[i])
            # print(tax)

    if customer_cmd_code_to_bvd.get(arr[2]) is None:
        customer_cmd_code_to_bvd[arr[2]] = tax
    else:
        customer_cmd_code_to_bvd[arr[2]] = customer_cmd_code_to_bvd[arr[2]].union(tax)

In [ ]:
no_bvd_num = 0
bvd_num_exists = 0
more_than_one_bvd_num = 0
# count = 10
for key, value in customer_cmd_code_to_bvd.items():
    if len(value) == 0:
        no_bvd_num += 1
    else:
        bvd_num_exists += 1
    if len(value) > 1:
        more_than_one_bvd_num += 1
    # count -= 1
    # if count == 0:
    #     break

In [ ]:
print(more_than_one_bvd_num)
print(no_bvd_num)
print(bvd_num_exists)

for now generating 3 types of corpus for comparison:
1. set of tax number
2. set of bvd number
3. dict of customer_cmd_code to list of entries corresponding to the same customer_cmd_code

In [135]:
import re
stop_words = {'llp', 'ltd', 'llc', 'plc', 'inc', 'corp', 'nv', 'group', 'gmbh', 'sa', 'company', 'corporation', 'limited'}
delimiters = {".", ",", "-", "_", "'", "&", "/", "\\", ":", ";", "(", ")", "[", "]", "{", "}", " "}
def split_company_name(name):
    pattern = '|'.join(map(re.escape, delimiters))
    words = re.split(pattern, name)
    return [word for word in words if word]

def lowercase_company_name(name):
    return name.lower()

# List of common delimiters
def remove_stop_words(words):
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def clean_company_name(trading_name):
    # Lowercase the company name
    trading_name = lowercase_company_name(trading_name)
    # Split the company name into words
    words = split_company_name(trading_name)
    # Remove stop words
    clean_name = remove_stop_words(words)
    return clean_name

In [136]:
max_number_matches = 5
from rapidfuzz import fuzz, process
import heapq
def sorted_tokens(name):
    return " ".join(sorted(name.split()))

#calculates the minimum number of edits (insertions, deletions, substitutions) needed to transform one string into another.
#For typos or minor spelling variations
def levenshtein_matching_score(input_name, name):
    levenshtein_matching_score = fuzz.ratio(input_name, name)
    return levenshtein_matching_score

#split strings into tokens (words) and compare the intersection and order of these tokens. This is useful for handling different word orders or missing words in one string.
def token_set_matching_score(input_name, name):
    token_set_matching_score = fuzz.token_set_ratio(input_name, name)
    return token_set_matching_score

#For situations with extra content or partial matches:
def partial_matching_score(input_name, name):
    partial_matching_score = fuzz.partial_token_set_ratio(input_name, name)
    return partial_matching_score

#focus on finding matching subsequences within strings. This is beneficial when comparing strings with extra content or typos.
#For typos or minor spelling variations
def substring_matching_score(input_name, name):
    substring_matching_score = fuzz.partial_ratio(input_name, name)
    return substring_matching_score

#split strings into tokens (words) and compare the intersection and order of these tokens. This is useful for handling different word orders or missing words in one string.
def token_sort_ratio_score(input_name, name):
    score = fuzz.token_sort_ratio(input_name, name)
    return score
# Matching

def match_name(input_name, corpus):
    input_name = clean_company_name(input_name)
    best_match = None
    best_score = 0
    top_matches = []
    heapq.heapify(top_matches)
    similarity_measures = { 
        "levenshtein_matching_score" : (0, levenshtein_matching_score),
        "token_set_matching_score" : (1, token_set_matching_score),
        "substring_matching_score" : (1, substring_matching_score),
        "partial_matching_score" : (1, partial_matching_score),
        "token_sort_ratio_score" : (0, token_sort_ratio_score)
    }
    for customer_info in corpus:
        name = customer_info[4]
        name = clean_company_name(name)
        num_measures = 0
        total_score = 0
        for function_check in similarity_measures.values():
            check, func = function_check
            if check:
                score = func(input_name, name)
                total_score += score
                num_measures +=1
            if num_measures == 0:
                combined_score = 0
            else:
                combined_score = total_score / num_measures
        if len(top_matches) < max_number_matches:
            heapq.heappush(top_matches, (combined_score, customer_info))
        else:
            heapq.heappushpop(top_matches, (combined_score, customer_info))
        if combined_score > best_score:
            best_score = combined_score
            best_match = customer_info
    
    return best_match, best_score, top_matches



In [137]:
#preparing data for testing
corpus_ratio = 0.9

num_customer_corpus = int(len(data) * corpus_ratio)

testing_customer_corpus = []

unique_tax_numbers = {}
unique_bvd_numbers = {}
corpus_customer_cmd_code = {}
corpus_legal_cluster_id = {}
country_wise_corpus = {}
for idx, arr in enumerate(data[1:num_customer_corpus]):
    country_name = arr[16].lower()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            if unique_bvd_numbers.get(arr[i].lower()):
                unique_bvd_numbers[arr[i].lower()].append(arr)
            else:
                unique_bvd_numbers[arr[i].lower()] = [arr]
            # unique_bvd_numbers[arr[i]] = arr
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            if unique_tax_numbers.get(arr[i].lower()):
                unique_tax_numbers[arr[i].lower()].append(arr)
            else:
                unique_tax_numbers[arr[i].lower()] = [arr]
            # unique_tax_numbers[arr[i]] = arr

    if country_wise_corpus.get(country_name):
        country_wise_corpus[country_name].append(arr)
    else:
        country_wise_corpus[country_name] = [arr]
    
    customer_cmd_code = arr[2].lower()
    if corpus_customer_cmd_code.get(customer_cmd_code):
        corpus_customer_cmd_code[customer_cmd_code].append(arr)
    else:    
        corpus_customer_cmd_code[customer_cmd_code] = [arr]
    
    legal_cluster_id = arr[1].lower()
    if corpus_legal_cluster_id.get(legal_cluster_id):
        corpus_legal_cluster_id[legal_cluster_id].append(arr)
    else:
        corpus_legal_cluster_id[legal_cluster_id] = [arr]

In [138]:
def extract_tax_numbers(arr):
    unique_tax_numbers = set()
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            unique_tax_numbers.add(arr[i].lower())
    return unique_tax_numbers

def extract_bvd_numbers(arr):
    unique_bvd_numbers = set()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            unique_bvd_numbers.add(arr[i].lower())
    return unique_bvd_numbers

In [139]:

def match_tax(arr):
    tax = extract_tax_numbers(arr)
    for tax_num in tax:
        if tax_num in unique_tax_numbers:
            return unique_tax_numbers[tax_num]
    return None

def match_bvd(arr):
    bvd = extract_bvd_numbers(arr)
    for bvd_num in bvd:
        if bvd_num in unique_bvd_numbers:
            return unique_bvd_numbers[bvd_num]
    return None

In [140]:
def reformat_top_matches(top_matches):
    matches = {}
    sorted_matches = sorted(top_matches, key=lambda x: x[0], reverse=True)
    rank = 1
    for idx, match in enumerate(sorted_matches):
        score = match[0]
        match_info = {
            "score" : score,
            "customer_info" : match[1]
        }
        matches[rank].append(match_info)
        rank += 1
        flag = False
        legal_cluster_id = match[1][1]
        # print(match)
        # print(legal_cluster_id)
        for r, existing_match in matches.items():
            if existing_match[0]["customer_info"][1] == legal_cluster_id:
                matches[r].append(match_info)
                flag = True
                break
        if not flag:
            matches[rank] = [match_info]
            rank += 1
    # print(matches)
    return matches

In [141]:
from tqdm import tqdm
success = 0
failure = 0
not_applicable = 0
result = {}
for arr in tqdm(data[num_customer_corpus:]):
    customer_cmd_code = arr[2].lower()
    # print(customer_cmd_code)
    legal_cluster_id = arr[1].lower()
    #checking if the test customer is in the corpus for the comparison
    if legal_cluster_id in corpus_legal_cluster_id.keys():
        matches = {}
        #tax matching
        tax_match = match_tax(arr)
        if tax_match:
            success += 1
            matches[1] = tax_match
            result[str(arr)] = matches
            continue
        #bvd matching
        bvd_match = match_bvd(arr)
        if bvd_match:
            success += 1
            matches[1] = bvd_match
            result[str(arr)] = matches
            continue

        #trading name matching after country filtering 
        country_name = arr[16].lower()
        #filtering by country and hence reducing our search space
        if country_wise_corpus.get(country_name) is None:
            not_applicable += 1
            continue
        filtered_corpus = country_wise_corpus[country_name]
        company_name = arr[4]
        cleaned_company_name = clean_company_name(company_name)
        best_match, best_score, top_matches = match_name(cleaned_company_name, filtered_corpus)
        reformatted_top_matches = reformat_top_matches(top_matches)
        # if legal_cluster_id == matched_company_info[0]:
        #     correct +=1
        # else:
        #     incorrect +=1
        if reformatted_top_matches.get(1):
            # print(reformatted_top_matches[1][0]['customer_info'][1])
            if legal_cluster_id == reformatted_top_matches[1][0]['customer_info'][1]:
                success +=1
            else:
                failure +=1
        
        matches = reformat_top_matches(top_matches)
        result[str(arr)] = matches
        
    else:
        not_applicable +=1    
        

 20%|██        | 73610/361060 [11:53:03<46:24:30,  1.72it/s]  


KeyboardInterrupt: 

In [ ]:
import json
with open('result_match_tax_bvd_name_country-2-3-4.json', 'w') as f:
    json.dump(result, f)
    f.close()

In [142]:
print(success, failure, not_applicable)

6083 2775 64752


In [143]:
success/(failure + success)

0.6867238654323775

In [ ]:
success + failure + not_applicable